In [10]:
# Calibration of Vasicek model to Stibor Market Rates

#Import necessary packages
import numpy as np
import scipy.interpolate as sci
from scipy.optimize import fmin


#Market data for calibration Updated with Stibor on 23 December 2020.
t_list = np.array((1, 7, 30, 60, 90, 180, 360)) / 360.
r_list =  np.array((-0.058, -0.099, -0.121, -0.106, -0.096, -0.053, -0.016)) / 100

#Calculate zero rates
factors = (1 + t_list * r_list)
zero_rates = 1 / t_list * np.log(factors)

#Today's rate 
r0 = r_list[0] # set to 1-day Stibor 

#Interpolation of Market Data              # Find the B-spline representation of a 1-D curve.
tck = sci.splrep(t_list, zero_rates, k=3)  # Cubic splines to interpolate. k = degree of spline fit.
tn_list = np.linspace(0.0, 1.0, 24)        
ts_list = sci.splev(tn_list, tck, der=0)   #Evaluate a B-spline or its derivatives
de_list = sci.splev(tn_list, tck, der=1)

#Forward rate transformation
f = ts_list + de_list * tn_list

# Vasicek model calulation of Forward Rates
def Vasicek_forward_rate(opt):
    kappa_r, theta_r, sigma_r = opt #mean-reversion factor, long-run mean, volatility factor = opt
    t = tn_list
    forward_rate = theta_r + np.exp(-kappa_r * t) * (r0 - theta_r) \
                - (sigma_r**2 / (2 * kappa_r**2)) * (1 - np.exp(- kappa_r * t))**2
    return forward_rate

#MSE loss function 
def Vasicek_error_function(opt):
    kappa_r, theta_r, sigma_r = opt
    if 2 * kappa_r * theta_r < sigma_r ** 2:
        return 100
    if theta_r < 0 or sigma_r < 0.001:
        return 100
    forward_rates = Vasicek_forward_rate(opt)
    MSE = np.sum((f - forward_rates) ** 2) / len(f)
    return MSE


#Calibration using downhill simplex algorithm
def Vasicek_calibration():
    opt = fmin(Vasicek_error_function, [1.0, 0.02, 0.1],
            xtol=0.00001, ftol=0.00001,
            maxiter=1000, maxfun=1000)
    return opt